In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [3]:
df = pd.read_csv("../data/dataset.csv", sep=";")
df.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,...,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,1,1,1,1,0,0,0,178839,9.653333,1.0
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,...,1,1,2,2,0,0,0,49014,13.181389,NaN
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1,1,2,2,0,0,0,124839,11.561944,1.0
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,0,0,0,324676,15.751111,1.0
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,0,1,1,1,0,0,0,7100,12.698611,NaN


In [5]:
df.set_index("uuid", inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99976 entries, 63f69b2c-8b1c-4740-b78d-52ed9a4515ac to ac88f18c-96a6-49bc-9e9d-a780225914af
Data columns (total 42 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   default                              89976 non-null  float64
 1   account_amount_added_12_24m          99976 non-null  int64  
 2   account_days_in_dc_12_24m            88140 non-null  float64
 3   account_days_in_rem_12_24m           88140 non-null  float64
 4   account_days_in_term_12_24m          88140 non-null  float64
 5   account_incoming_debt_vs_paid_0_24m  40661 non-null  float64
 6   account_status                       45603 non-null  float64
 7   account_worst_status_0_3m            45603 non-null  float64
 8   account_worst_status_12_24m          33215 non-null  float64
 9   account_worst_status_3_6m            42274 non-null  float64
 10  account_worst_status_6_12m       

In [9]:
test_df = df[df.default.isna()]

In [10]:
test_df.to_csv("../data/test_set.csv")

In [13]:
df.dropna(subset=["default"], inplace=True)
df.shape

(89976, 42)

In [18]:
df.isnull().sum().sort_values(ascending=False)/len(df)*100

worst_status_active_inv                69.507424
account_worst_status_12_24m            66.745577
account_worst_status_6_12m             60.363875
account_incoming_debt_vs_paid_0_24m    59.301369
account_worst_status_3_6m              57.724282
account_worst_status_0_3m              54.385614
account_status                         54.385614
avg_payment_span_0_3m                  49.326487
avg_payment_span_0_12m                 23.859696
num_active_div_by_paid_inv_0_12m       22.959456
num_arch_written_off_12_24m            18.121499
num_arch_written_off_0_12m             18.121499
account_days_in_rem_12_24m             11.873166
account_days_in_term_12_24m            11.873166
account_days_in_dc_12_24m              11.873166
sum_paid_inv_0_12m                      0.000000
sum_capital_paid_account_12_24m         0.000000
sum_capital_paid_account_0_12m          0.000000
time_hours                              0.000000
status_max_archived_0_24_months         0.000000
status_max_archived_

In [19]:
df.dtypes

default                                float64
account_amount_added_12_24m              int64
account_days_in_dc_12_24m              float64
account_days_in_rem_12_24m             float64
account_days_in_term_12_24m            float64
account_incoming_debt_vs_paid_0_24m    float64
account_status                         float64
account_worst_status_0_3m              float64
account_worst_status_12_24m            float64
account_worst_status_3_6m              float64
account_worst_status_6_12m             float64
age                                      int64
avg_payment_span_0_12m                 float64
avg_payment_span_0_3m                  float64
merchant_category                       object
merchant_group                          object
has_paid                                  bool
max_paid_inv_0_12m                     float64
max_paid_inv_0_24m                     float64
name_in_email                           object
num_active_div_by_paid_inv_0_12m       float64
num_active_in

In [ ]:
y = df.default

In [27]:
df.drop(columns="default", inplace=True)

KeyError: "['default'] not found in axis"

In [30]:
cat_features = df.select_dtypes("object").columns
cat_features

Index(['merchant_category', 'merchant_group', 'name_in_email'], dtype='object')

In [31]:
num_features = df.drop(columns=cat_features).columns
num_features

Index(['account_amount_added_12_24m', 'account_days_in_dc_12_24m',
       'account_days_in_rem_12_24m', 'account_days_in_term_12_24m',
       'account_incoming_debt_vs_paid_0_24m', 'account_status',
       'account_worst_status_0_3m', 'account_worst_status_12_24m',
       'account_worst_status_3_6m', 'account_worst_status_6_12m', 'age',
       'avg_payment_span_0_12m', 'avg_payment_span_0_3m', 'has_paid',
       'max_paid_inv_0_12m', 'max_paid_inv_0_24m',
       'num_active_div_by_paid_inv_0_12m', 'num_active_inv',
       'num_arch_dc_0_12m', 'num_arch_dc_12_24m', 'num_arch_ok_0_12m',
       'num_arch_ok_12_24m', 'num_arch_rem_0_12m',
       'num_arch_written_off_0_12m', 'num_arch_written_off_12_24m',
       'num_unpaid_bills', 'status_last_archived_0_24m',
       'status_2nd_last_archived_0_24m', 'status_3rd_last_archived_0_24m',
       'status_max_archived_0_6_months', 'status_max_archived_0_12_months',
       'status_max_archived_0_24_months', 'recovery_debt',
       'sum_capital_pa

In [42]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import OrdinalEncoder

In [35]:
df[cat_features].isnull().sum()

merchant_category    0
merchant_group       0
name_in_email        0
dtype: int64

In [37]:
df[num_features].isnull().sum().sort_values(ascending=False)

worst_status_active_inv                62540
account_worst_status_12_24m            60055
account_worst_status_6_12m             54313
account_incoming_debt_vs_paid_0_24m    53357
account_worst_status_3_6m              51938
account_status                         48934
account_worst_status_0_3m              48934
avg_payment_span_0_3m                  44382
avg_payment_span_0_12m                 21468
num_active_div_by_paid_inv_0_12m       20658
num_arch_written_off_12_24m            16305
num_arch_written_off_0_12m             16305
account_days_in_term_12_24m            10683
account_days_in_rem_12_24m             10683
account_days_in_dc_12_24m              10683
status_3rd_last_archived_0_24m             0
status_max_archived_0_6_months             0
status_max_archived_0_12_months            0
sum_paid_inv_0_12m                         0
status_max_archived_0_24_months            0
recovery_debt                              0
sum_capital_paid_account_0_12m             0
sum_capita

In [96]:
df.worst_status_active_inv.value_counts()

1.0    24230
2.0     3051
3.0      155
Name: worst_status_active_inv, dtype: int64

In [80]:
basic_preproc = make_column_transformer(
    (OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan), cat_features),
    remainder="passthrough"
)

In [86]:
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier()

In [87]:
basic_pipeline = make_pipeline(
    basic_preproc,
    model
)

In [88]:
basic_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=nan),
                                                  Index(['merchant_category', 'merchant_group', 'name_in_email'], dtype='object'))])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())])

In [89]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)

In [90]:
basic_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=nan),
                                                  Index(['merchant_category', 'merchant_group', 'name_in_email'], dtype='object'))])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())])

In [95]:
cross_val_score(basic_pipeline, X_test, y_test, scoring="recall").mean()

0.04912280701754386